## Initial Set up
Intitialize the necessary variables including a prime q, number of samples d, matrix a, errors e, and the secret.

In [7]:
d = 5
q = random_prime(2000, lbound=100)
Z37 = Integers(q)
Z3 = Integers(3)
a = []
e = []
    
for i in range(d):
    a.append(Z37.random_element())
    e.append(Integer((Z3.random_element()))-1)

ZX37.<s> = PolynomialRing(Z37)

secret = Z37.random_element()
    
print("secret:",secret,"\n")
print("prime:",q, "\n")
print("a:",a,"\n")
print("e:",e,"\n")

p = [Z37(0)] * d
prime = []
b = [Z37(0)] * d
newc = []

secret: 71 

prime: 127 

a: [89, 34, 102, 40, 84] 

e: [0, 0, 0, -1, 1] 



## Construct System of Equations
Compute each entry of b, then subtract out each possible error. Multiply these polynomials together to obtain a new polynomial that must be zero when the variable s = secret.

Put the coefficients in a matrix to prepare for Gaussian elimination (linearizes the variables - treats s^2 as it's own variable with no relation to s).

In [8]:
    for i in range(len(a)):
        #could it be here that the construction fails??
        b[i] = (a[i] * secret) + e[i]
        prime.append((a[i] * secret) + e[i])
        p1 = ZX37(b[i] - (a[i] * s))
        p2 = ZX37(b[i] - (a[i] * s + 1))
        p3 = ZX37(b[i] - (a[i] * s - 1))
        p[i] = (p1 * p2 * p3)
        print(p[i])
        
        c = []
        for coeff in p[i]: 
            c.append(coeff)
        if(c != []):
            c.reverse()
            #print(c)
            c.append(q-(c[len(c) - 1]))
            c[len(c) - 2 ] = 0
            #print(c)
            newc.append(c)

8*s^3 + 74*s^2 + 42*s + 85
66*s^3 + 39*s^2 + 59*s
4*s^3 + 37*s^2 + 15*s + 24
8*s^3 + 100*s^2 + 118*s + 21
5*s^3 + 37*s^2 + 116*s + 67


## Checks for correct construction

In [11]:
rows = len(newc)
    
M = MatrixSpace(Z37, rows, len(newc[0]))
M2 = MatrixSpace(Z37, rows, 1)
try:
    C = M(newc)
except ValueError:
    errors = errors + 1
    
cols = len(C[0])
    
M3 = MatrixSpace(Z37, cols, 1)
    
checker = []
for i in range(cols):
    checker.append(secret**(i))
checker = M3(checker.reverse())
zeros = M2()

## Final guess for the secret
After performing Gaussion elimination, we have solved for s, s^2 and s^3. This step checks whether s is correct and if the correct relationship between solutions is obtained.

In [12]:
print("c:\n",C,"\n")
C = C.echelon_form()
    
print(C, "\n")

u1 = C[2, cols-1]
u2 = C[1, cols-1]
u3 = C[0, cols-1]
    

if u1 == secret:
    #correct_secret = correct_secret + 1
    print('You guessed:', u1, "\n")
    print('The secret is:', secret, "\n")
else:
    print(u1, "\t", secret,"\n")

c:
 [  8  74  42   0  42]
[ 66  39  59   0   0]
[  4  37  15   0 103]
[  8 100 118   0 106]
[  5  37 116   0  60] 

[ 1  0  0  0 25]
[ 0  1  0  0 88]
[ 0  0  1  0 71]
[ 0  0  0  0  0]
[ 0  0  0  0  0] 

You guessed: 71 

The secret is: 71 



## LWE with secret size 3

Changes:
- Each row in the A matrix now has 3 columns
- The indeterminate used in the construction of the polynomials now has 3 distinct variables
- Using Groebner bases instead of Gaussian elimination

In [16]:
import random

correct_secret = 0
ones_or_thirty_six = 0
correct_relationship = 0
errors = 0
runs = 1

def num_samples(degree, n):
    return binomial(n + degree, degree)

for i in range(runs):
    #set up random variables
    d = num_samples(3, 3)
    q = random_prime(2000, lbound=100)
    Z37 = Integers(q)
    Z3 = Integers(3)
    a = []
    e = []
    
    S1 = MatrixSpace(Z37, 3, 1)
    A1 = MatrixSpace(Z37, 1, 3)
    
    for i in range(d):
        a.append(A1([Z37.random_element(), Z37.random_element(), Z37.random_element()]))
        e.append(Integer((Z3.random_element()))-1)
        
    ZX37.<s1, s2, s3> = PolynomialRing(Z37, order='degrevlex')
    
    indet = [s1, s2, s3]

    secret = S1([[Z37.random_element()], [Z37.random_element()], [Z37.random_element()]])
    
    print("secret: \n",secret,"\n")
    print("prime:",q, "\n")
    print("a:",a,"\n")
    print("e:",e,"\n")

    p = [Z37(0)] * d
    b = [Z37(0)] * d
    newc = []

    for i in range(len(a)):
        #could it be here that the construction fails??
        b[i] = (a[i] * secret) + e[i]
        #print(b[i] - (a[i][0][0] * s1 + a[i][0][1] * s2))
        p1 = ZX37((b[i] - (a[i][0][0] * s1 + a[i][0][1] * s2 + a[i][0][2] * s3))[0])
        #p1 = ZX37((b[i] - (a[i] * indet)))
        p2 = ZX37((b[i] - (a[i][0][0] * s1 + a[i][0][1] * s2 + a[i][0][2] * s3 + 1))[0])
        p3 = ZX37((b[i] - (a[i][0][0] * s1 + a[i][0][1] * s2 + a[i][0][2] * s3 - 1))[0])
        p[i] = (p1 * p2 * p3)

        if(p[i](secret.list()[0], secret.list()[1], secret.list()[2]) != 0):
            print("DUMMY")
    
    print(p[0])
    F = ideal(p)
    
    grob = F.groebner_basis()
    print("\nbasis:",grob,"\n")
    
    u1 = q - Integer(''.join(str(grob[0])[5:]))
    u2 = q - Integer(''.join(str(grob[1])[5:]))
    u3 = q - Integer(''.join(str(grob[2])[5:]))
    print("You guessed:", u1, u2, u3)
    print("Secret:\n",secret[0], secret[1], secret[2],"\n")
    print("----")
    

    if (u1 == secret.list()[0]) and (u2 == secret.list()[1]) and (u3 == secret.list()[2]):
        correct_secret = correct_secret + 1

    else:
        print(q - u1, "\t", secret)
    
print("Secret correctly guessed: \t\t\t",correct_secret * 1.0 / runs)

secret: 
 [454]
[843]
[772] 

prime: 1039 

a: [[1032  267  841], [ 70 131 906], [966 369 108], [171 708 575], [574 828 359], [366 530 133], [ 89 850 603], [338 140 865], [671  57 134], [689 253 354], [133 857 440], [833 215  62], [132 455 834], [942  68 322], [ 22 981 276], [909 981 614], [439 143 646], [154 929 669], [1024  934  548], [852 175 760]] 

e: [0, -1, 1, 0, 0, 1, -1, -1, 1, -1, -1, 0, -1, -1, -1, 0, 1, 1, 1, -1] 

343*s1^3 + 233*s1^2*s2 + 909*s1*s2^2 + 317*s2^3 + 14*s1^2*s3 + 1010*s1*s2*s3 + 182*s2^2*s3 + 396*s1*s3^2 + 332*s2*s3^2 + 23*s3^3 + 957*s1^2 + 912*s1*s2 + 1012*s2^2 + 853*s1*s3 + 1009*s2*s3 + 338*s3^2 + 983*s1 + 58*s2 + 494*s3 + 155

basis: [s1 + 585, s2 + 196, s3 + 267] 

You guessed: 454 843 772
Secret:
 (454) (843) (772) 

----
Secret correctly guessed: 			 1.00000000000000
